In [19]:

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import datetime

from statsmodels.tsa.vector_ar.var_model import VAR
import mlflow
from mlflow.entities import ViewType
import airflow
import os 
import gcsfs

os.environ["GCLOUD_PROJECT"] = "formation-mle-dev"
#defining the gcp path
gcs = gcsfs.GCSFileSystem(project='formation-mle-dev')
artefacts_bucket = 'dataset-projet-mle-var-forecasting'
folder_data = 'gcs/data'
folder_model = 'models'
gs_path = 'gs://'
bucket_name = artefacts_bucket
nom_df = "historic_demand_2009_2023_noNaN.csv"


tracking_server_host = "34.163.234.198"
mlflow.set_tracking_uri(f"http://{tracking_server_host}:5000")
mlflow.set_registry_uri('gs://mlflow-storage-artifacts-var-project')
mlflow.set_experiment("var_test_airflow")
experiment_name = "var_test_airflow"

if mlflow.get_experiment_by_name(experiment_name) is None:
    experiment_id=mlflow.create_experiment(experiment_name)
else:
    current_experiment=dict(mlflow.get_experiment_by_name(experiment_name))
    experiment_id=current_experiment['experiment_id']

run = mlflow.search_runs(
    experiment_ids=experiment_id,
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.MAE DESC"],
    output_format = 'pandas',
)
print(run)

                             run_id experiment_id    status  \
0  6a9247c5e1f6446ba2f8b8b1e5cdf3cf             3  FINISHED   

                                        artifact_uri  \
0  gs://mlflow-storage-artifacts-var-project/3/6a...   

                        start_time                         end_time  \
0 2023-05-26 09:35:54.175000+00:00 2023-05-26 09:38:38.850000+00:00   

   metrics.MAE  metrics.aic  metrics.df_model  metrics.bic  ...  params.ic  \
0  2837.300728   114.722499             577.0   117.668031  ...        aic   

   params.method params.trend params.verbose params.maxlags  \
0            ols            c          False             48   

                       tags.mlflow.log-model.history tags.mlflow.source.type  \
0  [{"run_id": "6a9247c5e1f6446ba2f8b8b1e5cdf3cf"...                   LOCAL   

      tags.mlflow.source.name tags.mlflow.user tags.mlflow.runName  
0  /opt/python3.8/bin/airflow          airflow    marvelous-ox-485  

[1 rows x 22 columns]


In [22]:
logged_model = os.path.join('runs:', run['run_id'][0], 'model' ).replace("\\","/")
print(logged_model)

# Load model as a PyFuncModel.
loaded_model = mlflow.statsmodels.load_model(logged_model)

runs:/6a9247c5e1f6446ba2f8b8b1e5cdf3cf/model


In [23]:
loaded_model

In [24]:
loaded_model.k_ar

48